<a href="https://colab.research.google.com/github/rishabhranawat/phi-2-csqa/blob/main/commonsense_qa_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers
!pip install trl
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-gzlae8xa
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-gzlae8xa
  Resolved https://github.com/huggingface/transformers to commit edb170238febf7fc3e3278ed5b9ca0b2c40c70e3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.37.0.dev0-py3-none-any.whl size=8339155 sha256=6586a544fa3dce2ac49801eb29444e5900b1a75ff8d174f94f548fe7d038925d

In [2]:
import torch
import datasets
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from datasets import load_dataset
from torch.utils.data import DataLoader
from collections import defaultdict
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [4]:
torch.set_default_device("cuda")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
    total_memory = torch.cuda.get_device_properties(0).total_memory
    available_memory = total_memory - torch.cuda.memory_allocated()
    print('Total Memory:', round(total_memory/1024**3,1), 'GB')
    print('Available Memory:', round(available_memory/1024**3,1), 'GB')

In [6]:
csqa = load_dataset("tau/commonsense_qa", split="train")

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

In [9]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
tokenizer.pad_token = tokenizer.eos_token

In [54]:
def construct_few_shot_examples(batch):
  few_shot_examples = "Beginning Of Examples\n"
  for question, choices, answer_key in zip(batch['question'], batch['choices'], batch['answerKey']):
      choices_text = "\n".join([f"{label}: {text}" for label, text in zip(choices['label'], choices['text'])])
      question_text = f"Instruct: Choose the correct option: {question} \n{choices_text}. \nOutput: {answer_key}. \n \n"
      few_shot_examples += question_text
  few_shot_examples += "\nEnd of Examples"
  return few_shot_examples

# Tokenize a batch of examples
def tokenize_batch(batch, few_shot_examples=''):
    questions = []
    for question, choices in zip(batch['question'], batch['choices']):
        choices_text = "\n".join([f"{label}: {text}" for label, text in zip(choices['label'], choices['text'])])
        question_text = f"{few_shot_examples} \nInstruct: Choose the correct option: {question} \n{choices_text}. \nOutput:"
        questions.append(question_text)
    tokenized_batch = tokenizer(questions, return_tensors='pt', return_attention_mask=False, padding=True, truncation=True, max_length=512)
    return tokenized_batch

# Process a batch of examples through the model
def model_output_batch(model, batch, few_shot_examples):
    tokenized_batch = tokenize_batch(batch, few_shot_examples).to("cuda")
    outputs = model.generate(**tokenized_batch, max_length=200)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [45]:
few_shot_example = construct_few_shot_examples({key: csqa[key][:5] for key in ['question', 'choices', 'answerKey']})

In [48]:
# Define batch size
batch_size = 8  # Adjust based on your GPU/CPU memory capacity

# Process dataset in batches
results = []
for i in range(0, 24, batch_size):
    print(f'Batch: {i}')
    batch = {key: csqa[key][i:i+batch_size] for key in ['question', 'choices', 'answerKey']}
    results.extend(model_output_batch(model, batch, few_shot_example))

Batch: 0


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1134: UserWarning: Input length of input_ids is 370, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


Batch: 8


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1134: UserWarning: Input length of input_ids is 374, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


Batch: 16


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1134: UserWarning: Input length of input_ids is 367, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [55]:
# Define batch size
batch_size = 8  # Adjust based on your GPU/CPU memory capacity

# Process dataset in batches
results = []
for i in range(0, 24, batch_size):
    print(f'Batch: {i}')
    batch = {key: csqa[key][i:i+batch_size] for key in ['question', 'choices', 'answerKey']}
    results.extend(model_output_batch(model, batch, ''))

Batch: 0
Batch: 8
Batch: 16


In [60]:
print(results[4])

 
Instruct: Choose the correct option: The fox walked from the city into the forest, what was it looking for? 
A: pretty flowers.
B: hen house
C: natural habitat
D: storybook
E: dense forest. 
Output:!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


### SFT

In [7]:
def formatting_func(batch):
  data = []
  for question, choices, answer_key in zip(batch['question'], batch['choices'], batch['answerKey']):
      choices_text = "\n".join([f"{label}: {text}" for label, text in zip(choices['label'], choices['text'])])
      question_text = f"Instruct: Choose the correct option: {question} \n{choices_text}. \nOutput: {answer_key}. \n \n"
      data.append(question_text)
  return data

In [36]:
csqa = load_dataset("tau/commonsense_qa", split="train")
# csqa = csqa.with_format("torch", device="cuda")
response_template = "Output:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [37]:
trainer = SFTTrainer(
    model,
    train_dataset=csqa,
    formatting_func=formatting_func,
    data_collator=collator
)
trainer.train()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:222: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/9741 [00:00<?, ? examples/s]

Step,Training Loss
500,0.000000
1000,0.000000
1500,0.000000
2000,0.000000
2500,0.000000
3000,0.000000
3500,0.000000


TrainOutput(global_step=3654, training_loss=0.0, metrics={'train_runtime': 1218.6672, 'train_samples_per_second': 23.979, 'train_steps_per_second': 2.998, 'total_flos': 3.213262511778816e+16, 'train_loss': 0.0, 'epoch': 3.0})

In [39]:
model_save_name = 'phi2_sft_csqa_0115.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
torch.save(model.state_dict(), path)

In [ ]:
# Define batch size
batch_size = 8  # Adjust based on your GPU/CPU memory capacity

# Process dataset in batches
results = []
for i in range(0, 24, batch_size):
    print(f'Batch: {i}')
    batch = {key: csqa[key][i:i+batch_size] for key in ['question', 'choices', 'answerKey']}
    results.extend(model_output_batch(model, batch, ''))